# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [5]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [6]:
from keras.models import Sequential
from keras.layers import Dense

# Baseline model
np.random.seed(123)
baseline_model = Sequential()


np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(Dense(100, activation='relu', input_shape=n_features))

# Hidden layer with 50 units
baseline_model.add(Dense(50, activation='relu'))

# Output layer
baseline_model.add(Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 1s 8ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [7]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(
    ss_X.fit_transform(X_train_numeric),
    columns=numeric_columns
)

# Transform validate and test data
X_val_scaled = pd.DataFrame(
    ss_X.transform(X_val_numeric),
    columns=numeric_columns
)

X_test_scaled = pd.DataFrame(
    ss_X.transform(X_test_numeric),
    columns=numeric_columns
)


# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [8]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [9]:
# Train the model
normalized_input_model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=150,
    validation_data=(X_val, y_val)
)


Epoch 1/150
33/33 [==============================] - 1s 8ms/step - loss: 708947357911032587616256.0000 - mse: 708947357911032587616256.0000 - val_loss: 334764980385796325376.0000 - val_mse: 334764980385796325376.0000
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan          
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 3ms

33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] -

33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 137/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [10]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [11]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(
    X_train,
    y_train_scaled,
    batch_size=32,
    epochs=150,
    validation_data=(X_val, y_val_scaled)
)


Epoch 1/150
33/33 [==============================] - 1s 7ms/step - loss: 0.5175 - mse: 0.5175 - val_loss: 0.1948 - val_mse: 0.1948
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2421 - mse: 0.2421 - val_loss: 0.1515 - val_mse: 0.1515
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1938 - mse: 0.1938 - val_loss: 0.1431 - val_mse: 0.1431
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1784 - mse: 0.1784 - val_loss: 0.1400 - val_mse: 0.1400
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1560 - mse: 0.1560 - val_loss: 0.1333 - val_mse: 0.1333
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1458 - mse: 0.1458 - val_loss: 0.1309 - val_mse: 0.1309
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1313 - mse: 0.1313 - val_loss: 0.1300 - val_mse: 0.1300
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1200 - m

33/33 [==============================] - 0s 3ms/step - loss: 0.0251 - mse: 0.0251 - val_loss: 0.1509 - val_mse: 0.1509
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0246 - mse: 0.0246 - val_loss: 0.1490 - val_mse: 0.1490
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0244 - mse: 0.0244 - val_loss: 0.1496 - val_mse: 0.1496
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0236 - mse: 0.0236 - val_loss: 0.1471 - val_mse: 0.1471
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0239 - mse: 0.0239 - val_loss: 0.1488 - val_mse: 0.1488
Epoch 68/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0229 - mse: 0.0229 - val_loss: 0.1489 - val_mse: 0.1489
Epoch 69/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0234 - mse: 0.0234 - val_loss: 0.1463 - val_mse: 0.1463
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0225 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0114 - mse: 0.0114 - val_loss: 0.1447 - val_mse: 0.1447
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.1457 - val_mse: 0.1457
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1452 - val_mse: 0.1452
Epoch 128/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0111 - mse: 0.0111 - val_loss: 0.1436 - val_mse: 0.1436
Epoch 129/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1456 - val_mse: 0.1456
Epoch 130/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1441 - val_mse: 0.1441
Epoch 131/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0107 - mse: 0.0107 - val_loss: 0.1452 - val_mse: 0.1452
Epoch 132/150
33/33 [==============================] - 0s 3ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [12]:
# Evaluate the model on training data
train_mse_scaled, train_mse_scaled_metric = normalized_model.evaluate(X_train, y_train_scaled)
print("Train MSE (scaled):", train_mse_scaled)


33/33 [==============================] - 0s 1ms/step - loss: 0.0083 - mse: 0.0083
Train MSE (scaled): 0.008286421187222004


- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [13]:
# Evaluate the model on validate data
val_mse_scaled, val_mse_scaled_metric = normalized_model.evaluate(X_val, y_val_scaled)
print("Validation MSE (scaled):", val_mse_scaled)


9/9 [==============================] - 0s 2ms/step - loss: 0.1458 - mse: 0.1458
Validation MSE (scaled): 0.1457958072423935


Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [14]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)


# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
val_rmse_original = mean_squared_error(y_val, y_val_pred, squared=False)
print("Validation RMSE (original scale):", val_rmse_original)


9/9 [==============================] - 0s 1ms/step
Validation RMSE (original scale): 30005.71717027349


Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [15]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(
    100,
    activation='relu',
    input_shape=n_features,
    kernel_initializer='he_normal'
))


# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(
    optimizer='SGD',
    loss='mse',
    metrics=['mse']
)

# Train the model
he_model.fit(
    X_train,
    y_train_scaled,
    batch_size=32,
    epochs=150,
    validation_data=(X_val, y_val_scaled)
)


Epoch 1/150
33/33 [==============================] - 1s 7ms/step - loss: 0.4861 - mse: 0.4861 - val_loss: 0.2404 - val_mse: 0.2404
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2567 - mse: 0.2567 - val_loss: 0.1931 - val_mse: 0.1931
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2053 - mse: 0.2053 - val_loss: 0.1648 - val_mse: 0.1648
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1785 - mse: 0.1785 - val_loss: 0.1543 - val_mse: 0.1543
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1556 - mse: 0.1556 - val_loss: 0.1489 - val_mse: 0.1489
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1421 - mse: 0.1421 - val_loss: 0.1447 - val_mse: 0.1447
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1291 - mse: 0.1291 - val_loss: 0.1397 - val_mse: 0.1397
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1179 - m

33/33 [==============================] - 0s 3ms/step - loss: 0.0241 - mse: 0.0241 - val_loss: 0.1308 - val_mse: 0.1308
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0240 - mse: 0.0240 - val_loss: 0.1236 - val_mse: 0.1236
Epoch 65/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0237 - mse: 0.0237 - val_loss: 0.1233 - val_mse: 0.1233
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0233 - mse: 0.0233 - val_loss: 0.1240 - val_mse: 0.1240
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0230 - mse: 0.0230 - val_loss: 0.1237 - val_mse: 0.1237
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0231 - mse: 0.0231 - val_loss: 0.1257 - val_mse: 0.1257
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0218 - mse: 0.0218 - val_loss: 0.1260 - val_mse: 0.1260
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0224 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.1275 - val_mse: 0.1275
Epoch 126/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1293 - val_mse: 0.1293
Epoch 127/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1298 - val_mse: 0.1298
Epoch 128/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0104 - mse: 0.0104 - val_loss: 0.1290 - val_mse: 0.1290
Epoch 129/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1274 - val_mse: 0.1274
Epoch 130/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1297 - val_mse: 0.1297
Epoch 131/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1279 - val_mse: 0.1279
Epoch 132/150
33/33 [==============================] - 0s 3ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [16]:
# Evaluate the model on training data
train_mse_he, _ = he_model.evaluate(X_train, y_train_scaled)
print("He model - Train MSE (scaled):", train_mse_he)


33/33 [==============================] - 0s 2ms/step - loss: 0.0076 - mse: 0.0076
He model - Train MSE (scaled): 0.007576348260045052


Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [17]:
# Evaluate the model on validate data
val_mse_he, _ = he_model.evaluate(X_val, y_val_scaled)
print("He model - Validation MSE (scaled):", val_mse_he)


9/9 [==============================] - 0s 2ms/step - loss: 0.1295 - mse: 0.1295
He model - Validation MSE (scaled): 0.1295318305492401


## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [18]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=n_features))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(
    optimizer='SGD',
    loss='mse',
    metrics=['mse']
)


# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 7ms/step - loss: 0.6392 - mse: 0.6392 - val_loss: 0.2248 - val_mse: 0.2248
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2793 - mse: 0.2793 - val_loss: 0.1471 - val_mse: 0.1471
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2191 - mse: 0.2191 - val_loss: 0.1354 - val_mse: 0.1354
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1793 - mse: 0.1793 - val_loss: 0.1113 - val_mse: 0.1113
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1675 - mse: 0.1675 - val_loss: 0.1069 - val_mse: 0.1069
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1493 - mse: 0.1493 - val_loss: 0.0992 - val_mse: 0.0992
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1376 - mse: 0.1376 - val_loss: 0.0979 - val_mse: 0.0979
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1298 - m

33/33 [==============================] - 0s 3ms/step - loss: 0.0240 - mse: 0.0240 - val_loss: 0.1125 - val_mse: 0.1125
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0237 - mse: 0.0237 - val_loss: 0.1139 - val_mse: 0.1139
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0235 - mse: 0.0235 - val_loss: 0.1128 - val_mse: 0.1128
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0231 - mse: 0.0231 - val_loss: 0.1137 - val_mse: 0.1137
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0223 - mse: 0.0223 - val_loss: 0.1137 - val_mse: 0.1137
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0220 - mse: 0.0220 - val_loss: 0.1158 - val_mse: 0.1158
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0217 - mse: 0.0217 - val_loss: 0.1176 - val_mse: 0.1176
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0215 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1181 - val_mse: 0.1181
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1173 - val_mse: 0.1173
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1183 - val_mse: 0.1183
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1176 - val_mse: 0.1176
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1167 - val_mse: 0.1167
Epoch 130/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0104 - mse: 0.0104 - val_loss: 0.1169 - val_mse: 0.1169
Epoch 131/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1179 - val_mse: 0.1179
Epoch 132/150
33/33 [==============================] - 0s 3ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on training data
train_mse_lecun, _ = lecun_model.evaluate(X_train, y_train_scaled)
print("Lecun model - Train MSE (scaled):", train_mse_lecun)


33/33 [==============================] - 0s 2ms/step - loss: 0.0081 - mse: 0.0081
Lecun model - Train MSE (scaled): 0.008146754465997219


Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [20]:
# Evaluate the model on validate data
val_mse_lecun, _ = lecun_model.evaluate(X_val, y_val_scaled)
print("Lecun model - Validation MSE (scaled):", val_mse_lecun)


9/9 [==============================] - 0s 2ms/step - loss: 0.1187 - mse: 0.1187
Lecun model - Validation MSE (scaled): 0.11871156096458435


Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [21]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(
    optimizer='rmsprop',
    loss='mse',
    metrics=['mse']
)


# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 7ms/step - loss: 0.3054 - mse: 0.3054 - val_loss: 0.1088 - val_mse: 0.1088
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1703 - mse: 0.1703 - val_loss: 0.1221 - val_mse: 0.1221
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1278 - mse: 0.1278 - val_loss: 0.0919 - val_mse: 0.0919
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1069 - mse: 0.1069 - val_loss: 0.1030 - val_mse: 0.1030
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0810 - mse: 0.0810 - val_loss: 0.1275 - val_mse: 0.1275
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0658 - mse: 0.0658 - val_loss: 0.1173 - val_mse: 0.1173
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0557 - mse: 0.0557 - val_loss: 0.1154 - val_mse: 0.1154
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0414 - m

33/33 [==============================] - 0s 3ms/step - loss: 0.0070 - mse: 0.0070 - val_loss: 0.1027 - val_mse: 0.1027
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1054 - val_mse: 0.1054
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0075 - mse: 0.0075 - val_loss: 0.1192 - val_mse: 0.1192
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.0884 - val_mse: 0.0884
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0096 - mse: 0.0096 - val_loss: 0.1075 - val_mse: 0.1075
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1291 - val_mse: 0.1291
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0069 - mse: 0.0069 - val_loss: 0.1007 - val_mse: 0.1007
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0087 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0077 - mse: 0.0077 - val_loss: 0.0993 - val_mse: 0.0993
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0039 - mse: 0.0039 - val_loss: 0.0911 - val_mse: 0.0911
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0048 - mse: 0.0048 - val_loss: 0.0985 - val_mse: 0.0985
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0080 - mse: 0.0080 - val_loss: 0.0858 - val_mse: 0.0858
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0046 - mse: 0.0046 - val_loss: 0.1012 - val_mse: 0.1012
Epoch 130/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0054 - mse: 0.0054 - val_loss: 0.0956 - val_mse: 0.0956
Epoch 131/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0070 - mse: 0.0070 - val_loss: 0.0927 - val_mse: 0.0927
Epoch 132/150
33/33 [==============================] - 0s 3ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on training data
train_mse_rmsprop, _ = rmsprop_model.evaluate(X_train, y_train_scaled)
print("RMSProp model - Train MSE (scaled):", train_mse_rmsprop)


33/33 [==============================] - 0s 1ms/step - loss: 0.0032 - mse: 0.0032
RMSProp model - Train MSE (scaled): 0.003229099791496992


Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [23]:
# Evaluate the model on validate data
val_mse_rmsprop, _ = rmsprop_model.evaluate(X_val, y_val_scaled)
print("RMSProp model - Validation MSE (scaled):", val_mse_rmsprop)


9/9 [==============================] - 0s 2ms/step - loss: 0.0934 - mse: 0.0934
RMSProp model - Validation MSE (scaled): 0.09337516129016876


## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [24]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mse']
)


# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 8ms/step - loss: 0.4017 - mse: 0.4017 - val_loss: 0.1370 - val_mse: 0.1370
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1540 - mse: 0.1540 - val_loss: 0.1114 - val_mse: 0.1114
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1046 - mse: 0.1046 - val_loss: 0.1142 - val_mse: 0.1142
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0795 - mse: 0.0795 - val_loss: 0.1161 - val_mse: 0.1161
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0582 - mse: 0.0582 - val_loss: 0.1243 - val_mse: 0.1243
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0429 - mse: 0.0429 - val_loss: 0.1153 - val_mse: 0.1153
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0334 - mse: 0.0334 - val_loss: 0.1188 - val_mse: 0.1188
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0293 - m

33/33 [==============================] - 0s 4ms/step - loss: 7.0647e-04 - mse: 7.0647e-04 - val_loss: 0.1121 - val_mse: 0.1121
Epoch 64/150
33/33 [==============================] - 0s 4ms/step - loss: 7.2907e-04 - mse: 7.2907e-04 - val_loss: 0.1096 - val_mse: 0.1096
Epoch 65/150
33/33 [==============================] - 0s 4ms/step - loss: 8.2895e-04 - mse: 8.2895e-04 - val_loss: 0.1144 - val_mse: 0.1144
Epoch 66/150
33/33 [==============================] - 0s 4ms/step - loss: 8.9694e-04 - mse: 8.9694e-04 - val_loss: 0.1065 - val_mse: 0.1065
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.1152 - val_mse: 0.1152
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0013 - mse: 0.0013 - val_loss: 0.1087 - val_mse: 0.1087
Epoch 69/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.1103 - val_mse: 0.1103
Epoch 70/150
33/33 [==============================] - 0s 

33/33 [==============================] - 0s 3ms/step - loss: 0.0203 - mse: 0.0203 - val_loss: 0.0965 - val_mse: 0.0965
Epoch 124/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0165 - mse: 0.0165 - val_loss: 0.0979 - val_mse: 0.0979
Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0119 - mse: 0.0119 - val_loss: 0.0948 - val_mse: 0.0948
Epoch 126/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0089 - mse: 0.0089 - val_loss: 0.1035 - val_mse: 0.1035
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0052 - mse: 0.0052 - val_loss: 0.0873 - val_mse: 0.0873
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0030 - mse: 0.0030 - val_loss: 0.0968 - val_mse: 0.0968
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0014 - mse: 0.0014 - val_loss: 0.0908 - val_mse: 0.0908
Epoch 130/150
33/33 [==============================] - 0s 3ms/step - loss: 7.3418e-

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on training data
train_mse_adam, _ = adam_model.evaluate(X_train, y_train_scaled)
print("Adam model - Train MSE (scaled):", train_mse_adam)


33/33 [==============================] - 0s 2ms/step - loss: 1.1372e-04 - mse: 1.1372e-04
Adam model - Train MSE (scaled): 0.00011372280278010294


Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [26]:
# Evaluate the model on validate data
val_mse_adam, _ = adam_model.evaluate(X_val, y_val_scaled)
print("Adam model - Validation MSE (scaled):", val_mse_adam)


9/9 [==============================] - 0s 2ms/step - loss: 0.0963 - mse: 0.0963
Adam model - Validation MSE (scaled): 0.09633748978376389


## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [27]:
# Evaluate the best model on test data
test_mse_scaled, _ = adam_model.evaluate(X_test, y_test_scaled)
print("Test MSE (scaled):", test_mse_scaled)


5/5 [==============================] - 0s 2ms/step - loss: 0.1509 - mse: 0.1509
Test MSE (scaled): 0.1508716493844986


As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [28]:
# Generate predictions on test data
y_test_pred_scaled = adam_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
print("Test RMSE (original scale):", test_rmse)


5/5 [==============================] - 0s 2ms/step
Test RMSE (original scale): 30523.5685513734


## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.